In [2]:
pip install leafmap

  Using cached leafmap-0.46.5-py2.py3-none-any.whl.metadata (16 kB)
  Using cached anywidget-0.9.18-py3-none-any.whl.metadata (8.9 kB)
  Using cached bqplot-0.12.45-py2.py3-none-any.whl.metadata (6.4 kB)
  Using cached colour-0.1.5-py2.py3-none-any.whl.metadata (18 kB)
  Using cached duckdb-1.3.0-cp313-cp313-win_amd64.whl.metadata (7.2 kB)
  Using cached gdown-5.2.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached geojson-3.2.0-py3-none-any.whl.metadata (16 kB)
  Using cached h5netcdf-1.6.1-py3-none-any.whl.metadata (13 kB)
  Using cached h5py-3.13.0-cp313-cp313-win_amd64.whl.metadata (2.5 kB)
  Using cached ipyevents-2.0.2-py3-none-any.whl.metadata (2.9 kB)
  Using cached ipyfilechooser-0.6.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached ipyleaflet-0.19.2-py3-none-any.whl.metadata (5.3 kB)
  Using cached ipyvuetify-1.11.1-py2.py3-none-any.whl.metadata (7.5 kB)
  Using cached localtileserver-0.10.6-py3-none-any.whl.metadata (5.2 kB)
  Using cached opera_utils-0.21.0-py3-none-any.w

In [15]:
import requests
import pandas as pd
import geopandas as gpd
from shapely import wkt
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

# -----------------------------
# Utility Function to Read API Responses
# -----------------------------
def read_response(response):
    try:
        data = response.json()
        return data
    except requests.exceptions.JSONDecodeError:
        print("Invalid JSON response!")
        print("Response content:", response.text)
        return None

# -----------------------------
# Get Province Geometry as GeoDataFrame (optional, utility)
# -----------------------------
def get_province_shape():
    response = requests.get(url="http://127.0.0.1:5000/api/province_shape")
    data = read_response(response)
    if data:
        gdf = gpd.GeoDataFrame(data)
        gdf['geometry'] = gdf['geometry'].apply(wkt.loads)
        gdf.set_geometry('geometry', inplace=True)
        return gdf
    return gpd.GeoDataFrame()

# -----------------------------
# Get List of All Provinces
# -----------------------------
def get_provinces_list():
    response = requests.get("http://127.0.0.1:5000/api/provinces")
    data = read_response(response)
    return data if data else []

# -----------------------------
# Get List of All Municipalities
# -----------------------------
def get_municipalities_list():
    response = requests.get("http://127.0.0.1:5000/api/municipalities")
    data = read_response(response)
    return data if data else []

## DV-7 Pollutant comparison over same area
The interface allow the user to select one province or one municipality and visualize the time series of each pollutant, the time series will be obtained calculating for each pollutant the daily average over the selected area.

In [16]:
# Get DV-7 Data for Municipality
# -----------------------------
def get_dv7_municipality(municipality):
    response = requests.post(
        url="http://127.0.0.1:5000/api/DV_7comune",
        json={"var_comune": municipality}
    )
    if response.ok:
        return response.json()
    else:
        print("Error calling DV_7comune API")
        return {}

# -----------------------------
# Plot Time Series for Pollutants
# -----------------------------
def plot_time_series(data):
    clear_output(wait=True)
    if not data:
        print("No data available for the selected area.")
        return

    plt.figure(figsize=(12, 6))
    for pollutant, records in data.items():
        dates = [r["date"] for r in records]
        values = [r["value"] for r in records]
        plt.plot(dates, values, label=pollutant)
    plt.xlabel("Date")
    plt.ylabel("Average Daily Value")
    plt.title("DV-7 Pollutants Time Series")
    plt.legend()
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.show()

# -----------------------------
# UI Components
# -----------------------------
btn_province = widgets.Button(description="Select Province")
btn_municipality = widgets.Button(description="Select Municipality")

province_dropdown = widgets.Dropdown(options=[])
municipality_dropdown = widgets.Dropdown(options=[])

output = widgets.Output()

# -----------------------------
# Button Handlers
# -----------------------------
def on_btn_province_clicked(b):
    with output:
        clear_output()
        provinces = get_provinces_list()
        if not provinces:
            print("Could not load provinces list.")
            return
        province_dropdown.options = provinces
        display(province_dropdown)

def on_province_selected(change):
    if change['type'] == 'change' and change['name'] == 'value':
        selected_province = change['new']
        with output:
            clear_output()
            print(f"Loading data for province: {selected_province} ...")
            # Qui puoi aggiungere la chiamata per i dati DV-7 provincia se implementata
            print("Province-level DV-7 data API not implemented yet.")

def on_btn_municipality_clicked(b):
    with output:
        clear_output()
        municipalities = get_municipalities_list()
        if not municipalities:
            print("Could not load municipalities list.")
            return
        municipality_dropdown.options = municipalities
        display(municipality_dropdown)

def on_municipality_selected(change):
    if change['type'] == 'change' and change['name'] == 'value':
        selected_municipality = change['new']
        with output:
            clear_output()
            print(f"Loading data for municipality: {selected_municipality} ...")
            data = get_dv7_municipality(selected_municipality)
            plot_time_series(data)

# -----------------------------
# Link widgets and handlers
# -----------------------------
btn_province.on_click(on_btn_province_clicked)
province_dropdown.observe(on_province_selected)

btn_municipality.on_click(on_btn_municipality_clicked)
municipality_dropdown.observe(on_municipality_selected)

# -----------------------------
# Display UI
# -----------------------------
display(btn_province, btn_municipality, output)

Button(description='Select Province', style=ButtonStyle())

Button(description='Select Municipality', style=ButtonStyle())

Output()